In [1]:
import requests
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import os

# ============================================
# 0. 기본 설정
# ============================================
API_KEY = "e162c669fcbfdacee808cdff73d64f1a"
BASE_URL = "https://api.themoviedb.org/3"

# 반기(H1/H2) 기간 5개
PERIODS = [
    ("2023_H1", "2023-01-01", "2023-06-30"),
    ("2023_H2", "2023-07-01", "2023-12-31"),
    ("2024_H1", "2024-01-01", "2024-06-30"),
    ("2024_H2", "2024-07-01", "2024-12-31"),
    ("2025_H1", "2025-01-01", "2025-06-30"),
]

# discover에서 사용할 정렬 기준들
DISCOVER_SORT_OPTIONS = [
    "popularity.desc",
    "popularity.asc",
    "vote_count.desc",
    "vote_average.desc",  # + vote_count.gte를 같이 줄 예정
]

# 각 정렬 기준당 뽑을 페이지 수 (조절 가능)
PAGES_PER_SORT = 150   # 150 * 4 sort ≈ 600페이지/반기
MAX_DISCOVER_PAGE = 500    # TMDB 정책상 최대 500

# 상세 정보 수집 스레드 수
DETAIL_MAX_WORKERS = 15

# 최종 출력 CSV
OUTPUT_CSV = "tmdb_movies_2023_2025H1_full.csv"


# ============================================
# 공통: 안전한 요청 함수 (에러 + 레이트리밋 대응)
# ============================================
def safe_get(url, params, max_retries=3, base_wait=2):
    """
    requests.get에 레이트리밋(429) & 네트워크 오류 대비용 래퍼.
    - 429면 Retry-After 헤더 또는 base_wait초 대기 후 재시도
    - 그 외 에러는 점점 대기시간 늘리면서 재시도
    """
    for attempt in range(1, max_retries + 1):
        try:
            resp = requests.get(url, params=params, timeout=10)
            # 레이트 리밋
            if resp.status_code == 429:
                wait = int(resp.headers.get("Retry-After", base_wait))
                print(f"[RATE LIMIT] {wait}초 대기 후 재시도... (attempt={attempt})")
                time.sleep(wait)
                continue

            resp.raise_for_status()
            return resp

        except Exception as e:
            print(f"[REQUEST ERROR] attempt={attempt}, error={e}")
            if attempt == max_retries:
                return None
            # 점점 늘려서 대기
            time.sleep(base_wait * attempt)

    return None


# ============================================
# 1. 반기별 discover로 movie_id 모으기
# ============================================
def get_total_pages_for_period(start_date, end_date, sort_by):
    """
    해당 기간 + 정렬 기준으로 discover 호출해서 total_pages 확인
    """
    params = {
        "api_key": API_KEY,
        "language": "en-US",
        "sort_by": sort_by,
        "primary_release_date.gte": start_date,
        "primary_release_date.lte": end_date,
        "include_adult": "true",
        "include_video": "false",
        "page": 1,
    }
    # 평점 정렬일 때는 최소 투표 수 조건 살짝 추가해주면 이상치 덜함
    if sort_by.startswith("vote_average"):
        params["vote_count.gte"] = 20

    resp = safe_get(f"{BASE_URL}/discover/movie", params=params)
    if resp is None:
        return 0

    data = resp.json()
    return int(data.get("total_pages", 0))


def discover_ids_for_period(label, start_date, end_date):
    """
    한 반기(period)에 대해:
    - 여러 정렬 기준으로
    - 랜덤 페이지를 골라
    - movie_id를 최대한 많이 모은다.
    """
    collected_ids = set()

    print(f"\n===== [{label}] ID 수집 시작 =====")
    for sort_by in DISCOVER_SORT_OPTIONS:
        total_pages = get_total_pages_for_period(start_date, end_date, sort_by)
        if total_pages == 0:
            print(f"[{label}] sort_by={sort_by} total_pages=0 (건너뜀)")
            continue

        # TMDB 정책상 최대 500까지
        total_pages = min(total_pages, MAX_DISCOVER_PAGE)

        # 이 정렬 기준에서 사용할 페이지 리스트 (랜덤 샘플링)
        n_pages = min(PAGES_PER_SORT, total_pages)
        pages = random.sample(range(1, total_pages + 1), n_pages)

        print(f"[{label}] sort_by={sort_by}, total_pages={total_pages}, 샘플링 {n_pages}페이지")

        for page in pages:
            params = {
                "api_key": API_KEY,
                "language": "en-US",
                "sort_by": sort_by,
                "primary_release_date.gte": start_date,
                "primary_release_date.lte": end_date,
                "include_adult": "true",
                "include_video": "false",
                "page": page,
            }
            if sort_by.startswith("vote_average"):
                params["vote_count.gte"] = 20

            resp = safe_get(f"{BASE_URL}/discover/movie", params=params)
            if resp is None:
                print(f"[{label}] page={page} 요청 실패, 건너뜀")
                continue

            data = resp.json()
            results = data.get("results", [])
            for item in results:
                mid = item.get("id")
                if mid:
                    collected_ids.add(mid)

        print(f"[{label}] sort_by={sort_by} 완료, 누적 ID 수: {len(collected_ids)}")

    print(f"[{label}] 최종 유니크 ID 수: {len(collected_ids)}")
    return collected_ids


def collect_all_ids_for_periods():
    """
    PERIODS 전체에 대해 ID 모으고,
    최종 유니크 ID 리스트 반환
    """
    all_ids = set()
    period_id_dict = {}

    for label, start_date, end_date in PERIODS:
        ids = discover_ids_for_period(label, start_date, end_date)
        period_id_dict[label] = ids
        all_ids.update(ids)

    print("\n===== 전체 기간 기준 ID 요약 =====")
    for label, ids in period_id_dict.items():
        print(f"{label}: {len(ids)}개")
    print(f"전체 유니크 ID 개수: {len(all_ids)}")

    return list(all_ids)


# ============================================
# 2. 상세 정보 + credits + keywords + providers 수집
# ============================================
def join_list(items, key=None):
    """리스트를 '|'로 이어붙이는 유틸 함수"""
    if not items:
        return None
    if key:
        vals = [str(x.get(key, "")).strip() for x in items if x.get(key)]
    else:
        vals = [str(x).strip() for x in items if x]
    vals = [v for v in vals if v]
    return "|".join(vals) if vals else None


def parse_movie_to_row(data: dict, region: str = "KR") -> dict:
    """TMDB movie + credits + keywords + watch/providers 응답을
    표준 35컬럼 스키마 dict로 변환"""

    movie_id = data.get("id")
    imdb_id = data.get("imdb_id")
    title = data.get("title")
    original_title = data.get("original_title")
    original_language = data.get("original_language")
    overview = data.get("overview")
    tagline = data.get("tagline")
    homepage = data.get("homepage")
    status = data.get("status")
    release_date = data.get("release_date")
    runtime = data.get("runtime")
    adult = data.get("adult", False)
    budget = data.get("budget") or 0
    revenue = data.get("revenue") or 0
    vote_count = data.get("vote_count") or 0
    vote_average = data.get("vote_average") or 0.0
    popularity = data.get("popularity") or 0.0

    # release_year
    try:
        release_year = int(release_date[:4]) if release_date else None
    except Exception:
        release_year = None

    # roi
    roi = None
    if budget and revenue and budget > 0:
        roi = revenue / budget

    # 장르
    genres_list = data.get("genres", [])
    genres = join_list(genres_list, key="name")
    genre_ids_vals = [str(g.get("id")) for g in genres_list if g.get("id") is not None]
    genre_ids = "|".join(genre_ids_vals) if genre_ids_vals else None
    genre_count = len(genres_list) if genres_list else 0

    # 키워드
    keywords_data = data.get("keywords", {})
    keywords_list = keywords_data.get("keywords", [])
    keywords = join_list(keywords_list, key="name")
    keyword_count = len(keywords_list) if keywords_list else 0

    # 제작/국가/언어
    production_companies = join_list(data.get("production_companies", []), key="name")
    production_countries = join_list(data.get("production_countries", []), key="name")
    spoken_languages = join_list(data.get("spoken_languages", []), key="name")

    # 크레딧
    credits = data.get("credits", {})
    cast = credits.get("cast", [])
    crew = credits.get("crew", [])

    cast_sorted = sorted(cast, key=lambda x: x.get("order", 9999))
    top_cast_names = [c.get("name") for c in cast_sorted[:5] if c.get("name")]
    top_cast = "|".join(top_cast_names) if top_cast_names else None
    cast_count = len(cast) if cast else 0

    directors = [c.get("name") for c in crew if c.get("job") == "Director" and c.get("name")]
    director = "|".join(directors) if directors else None

    writers_list = [
        c.get("name")
        for c in crew
        if c.get("department") == "Writing" and c.get("name")
    ]
    writers = "|".join(sorted(set(writers_list))) if writers_list else None

    producers_list = [
        c.get("name")
        for c in crew
        if c.get("job") in ["Producer", "Executive Producer"] and c.get("name")
    ]
    producers = "|".join(sorted(set(producers_list))) if producers_list else None

    # OTT
    providers_data = data.get("watch/providers", {}).get("results", {})
    region_info = providers_data.get(region, {}) if providers_data else {}
    providers_flatrate = join_list(region_info.get("flatrate", []), key="provider_name")
    providers_rent = join_list(region_info.get("rent", []), key="provider_name")
    providers_buy = join_list(region_info.get("buy", []), key="provider_name")

    row = {
        "id": movie_id,
        "imdb_id": imdb_id,
        "title": title,
        "original_title": original_title,
        "original_language": original_language,
        "overview": overview,
        "tagline": tagline,
        "homepage": homepage,
        "status": status,
        "release_date": release_date,
        "release_year": release_year,
        "runtime": runtime,
        "adult": int(bool(adult)),
        "budget": budget,
        "revenue": revenue,
        "roi": roi,
        "vote_count": vote_count,
        "vote_average": vote_average,
        "popularity": popularity,
        "genres": genres,
        "genre_ids": genre_ids,
        "genre_count": genre_count,
        "keywords": keywords,
        "keyword_count": keyword_count,
        "production_companies": production_companies,
        "production_countries": production_countries,
        "spoken_languages": spoken_languages,
        "top_cast": top_cast,
        "director": director,
        "writers": writers,
        "producers": producers,
        "cast_count": cast_count,
        "providers_flatrate": providers_flatrate,
        "providers_rent": providers_rent,
        "providers_buy": providers_buy,
    }
    return row


def fetch_movie_detail(movie_id: int, region: str = "KR"):
    """단일 movie_id에 대해 상세 정보 요청"""
    params = {
        "api_key": API_KEY,
        "language": "en-US",
        "append_to_response": "credits,keywords,watch/providers",
    }
    resp = safe_get(f"{BASE_URL}/movie/{movie_id}", params=params)
    if resp is None:
        print(f"[ERROR] movie_id={movie_id} 요청 실패")
        return None

    try:
        data = resp.json()
        row = parse_movie_to_row(data, region=region)
        return row
    except Exception as e:
        print(f"[ERROR] movie_id={movie_id} 파싱 실패: {e}")
        return None


def collect_details_for_ids(movie_ids, max_workers=DETAIL_MAX_WORKERS):
    """
    movie_id 리스트에 대해 멀티스레드로 상세 정보 수집
    """
    rows = []
    total = len(movie_ids)
    print(f"\n===== 상세 정보 수집 시작 (총 {total}개) =====")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_movie_detail, mid): mid for mid in movie_ids}

        for i, f in enumerate(as_completed(futures), start=1):
            mid = futures[f]
            result = f.result()
            if result is not None:
                rows.append(result)
            if i % 100 == 0 or i == total:
                print(f"[detail] {i}/{total} 처리 완료 (누적 유효 {len(rows)}개)")

    print(f"[detail] 최종 유효 레코드 수: {len(rows)}")
    return rows


# ============================================
# 3. 메인 파이프라인
# ============================================
def main():
    # 1) 반기별 discover로 ID 전부 모으기
    movie_ids = collect_all_ids_for_periods()
    print(f"\n[main] 전체 유니크 movie_id 개수: {len(movie_ids)}")

    # 필요하면 여기서 샘플만 뽑아서 테스트도 가능:
    # movie_ids = movie_ids[:2000]

    # 2) 멀티스레드로 상세 정보 수집
    rows = collect_details_for_ids(movie_ids, max_workers=DETAIL_MAX_WORKERS)

    # 3) DataFrame으로 만들고 CSV 저장
    df = pd.DataFrame(rows)
    print(df.head())
    print(df.shape)
    print(df.columns)

    df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print(f'[main] "{OUTPUT_CSV}" 저장 완료')


if __name__ == "__main__":
    main()



===== [2023_H1] ID 수집 시작 =====
[2023_H1] sort_by=popularity.desc, total_pages=500, 샘플링 150페이지
[2023_H1] sort_by=popularity.desc 완료, 누적 ID 수: 3000
[2023_H1] sort_by=popularity.asc, total_pages=500, 샘플링 150페이지
[2023_H1] sort_by=popularity.asc 완료, 누적 ID 수: 6000
[2023_H1] sort_by=vote_count.desc, total_pages=500, 샘플링 150페이지
[2023_H1] sort_by=vote_count.desc 완료, 누적 ID 수: 8188
[2023_H1] sort_by=vote_average.desc, total_pages=41, 샘플링 41페이지
[2023_H1] sort_by=vote_average.desc 완료, 누적 ID 수: 8563
[2023_H1] 최종 유니크 ID 수: 8563

===== [2023_H2] ID 수집 시작 =====
[2023_H2] sort_by=popularity.desc, total_pages=500, 샘플링 150페이지
[2023_H2] sort_by=popularity.desc 완료, 누적 ID 수: 3000
[2023_H2] sort_by=popularity.asc, total_pages=500, 샘플링 150페이지
[2023_H2] sort_by=popularity.asc 완료, 누적 ID 수: 6000
[2023_H2] sort_by=vote_count.desc, total_pages=500, 샘플링 150페이지
[2023_H2] sort_by=vote_count.desc 완료, 누적 ID 수: 8231
[2023_H2] sort_by=vote_average.desc, total_pages=47, 샘플링 47페이지
[2023_H2] sort_by=vote_average.desc 완료, 누적

NameError: name 'df' is not defined